# SIGNOR Pipeline

This notebook creates a Network Set of SIGNOR pathway networks using the latest data downloaded via the SIGNOR REST service. SIGNOR is updated regularly and so it also demonstrates how to update the networks in the set.

## Imports Python Packages Needed in the Pipeline

In [1]:
import ndex2.client as nc
import json
import pandas as pd
import sys
import jsonschema
import ndexutil.tsv.tsv2nicecx as t2n
from tutorial_utils import load_tutorial_config

### Set variables

In [2]:
#cytoscape_visual_properties_template_id = 'ece36fa0-1e5d-11e8-b939-0ac135e8bacf' # PUBLIC
cytoscape_visual_properties_template_id = 'c7075eb1-231e-11e8-894b-525400c25d22' # DEV

my_server, my_username, my_password = load_tutorial_config("main")
# alternatively, edit and uncomment these lines to set the connection parameters manually
# my_server = "public.ndexbio.org"
# my_username = None
# my_password = None

my_ndex=nc.Ndex2(my_server, my_username, my_password)

### Set load plan

In [3]:
path_to_load_plan = 'hitpredict_load_plan.json'
load_plan = None
with open(path_to_load_plan, 'r') as lp:
    load_plan = json.load(lp)

### Process Hit Predict network

In [4]:
def process_hitpredict_network(load_plan):
    # TODO - add context (normalize?)
    # @CONTEXT is set from the load plan

    with open('HitPredit_in_KEGG.txt', 'rU') as tsvfile:
        header = [h.strip() for h in tsvfile.readline().split('\t')]

        df = pd.read_csv(tsvfile, delimiter='\t', na_filter=False, engine='python', names=header)

    # upcase column names
    rename = {}
    for column_name in df.columns:
        rename[column_name] = column_name.upper()

    df = df.rename(columns=rename)

    # IF THE NAME FIELD IS EMPTY - SUBSTITUTE UNIPROT FIELD
    df['NAME1'].replace('', df['UNIPROT1'], inplace=True)
    df['NAME2'].replace('', df['UNIPROT2'], inplace=True)

    # ADD COLUMN TO DATAFRAME TO BE USED AS DEFAULT INTERACTION
    df.loc[:, 'DEFAULT INTERACTION'] = pd.Series('interacts with', index=df.index)

    network = t2n.convert_pandas_to_nice_cx_with_load_plan(df, load_plan)

    for node_id, node in network.nodes.items():
        # IF NODE NAME HAS DELIMITER (;) THEN SPLIT AND TAKE FIRST ELEMENT
        if ';' in node.get_name():
            node_name_temp = node.get_name().split(';')
            node.set_node_name(node_name_temp[0])

        # STRIP OFF EXTRA DATA IN THE ALIAS ATTRIBUTE
        values = network.get_node_attribute(node, 'alias2')
        if not isinstance(values, list):
            values = [values]

        replacement_values = []
        for val in values:
            if val is None:
                break
            sub_values = val.split('[')
            replacement_values.append(sub_values[0])

        if len(replacement_values) < 1:
            network.remove_node_attribute(node, 'alias2')
        else:
            network_att = network.get_node_attribute_objects(node_id, 'alias2')
            network_att.set_values(replacement_values)

    network.set_network_attribute("organism", "Human, 9606, Homo sapiens")
    network.union_node_attributes('alias', 'alias2', 'alias')
    network.set_name('Hit Predict draft 2.0')

    network.apply_template(username=my_username, password=my_password, server=my_server,
                           uuid=cytoscape_visual_properties_template_id)
    message = network.upload_to(my_server, my_username, my_password)

print('starting...')
process_hitpredict_network(load_plan)
print('finished...')



starting...


/Users/aarongary/Development/Projects/ndex-example-development/vpy36/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 'U' mode is deprecated
  """


[{'name': 'provenanceHistory', 'elementCount': 1, 'version': '1.0', 'consistencyGroup': 1, 'properties': []}, {'name': 'nodes', 'elementCount': 55, 'idCounter': 2052, 'version': '1.0', 'consistencyGroup': 1, 'properties': []}, {'name': 'edges', 'elementCount': 54, 'idCounter': 4563, 'version': '1.0', 'consistencyGroup': 1, 'properties': []}, {'name': 'networkAttributes', 'elementCount': 4, 'version': '1.0', 'consistencyGroup': 1, 'properties': []}, {'name': 'nodeAttributes', 'elementCount': 55, 'version': '1.0', 'consistencyGroup': 1, 'properties': []}, {'name': 'edgeAttributes', 'elementCount': 540, 'version': '1.0', 'consistencyGroup': 1, 'properties': []}, {'name': 'cartesianLayout', 'elementCount': 55, 'version': '1.0', 'consistencyGroup': 1, 'properties': []}, {'name': 'cyVisualProperties', 'elementCount': 3, 'version': '1.0', 'consistencyGroup': 1, 'properties': []}, {'name': '@context', 'elementCount': 1, 'version': '1.0', 'consistencyGroup': 2, 'properties': []}]
finished...
